
# Implementação e Avaliação do FixMatch com CIFAR-10

### Objetivo
Neste assignment, vocês implementarão o método FixMatch, uma técnica de aprendizado semi-supervisionado que combina aprendizado supervisionado e não supervisionado. O objetivo é aplicar o FixMatch ao dataset CIFAR-10 para treinar uma rede neural e avaliar os resultados obtidos com diferentes proporções de dados rotulados.

### 1. Introdução ao FixMatch
FixMatch é uma técnica que combina pseudo-rotulagem e consistência de dados aumentados. Em resumo, o método:
- **Gera pseudo-rótulos** para dados não rotulados, utilizando uma predição confiável de dados fracamente aumentados.
- **Aplica uma consistência de pseudo-rótulos**, onde a rede é treinada para produzir as mesmas previsões em versões fortemente aumentadas das mesmas imagens.

Para mais informações sobre a arquitetura e a metodologia do FixMatch, vocês podem consultar o [paper original](https://arxiv.org/abs/2001.07685) e/ou ver os slides disponibilizados.

### 2. Estrutura da Implementação

1. **Dataset e Preparação dos Dados**  
   - Use o CIFAR-10 como dataset.
   - Prepare duas versões dos dados:
     - **Dados rotulados:** Utilizem um subconjunto rotulado do CIFAR-10 com diferentes quantidades de rótulos por classe para experimentação.
     - **Dados não rotulados:** O restante do CIFAR-10 deve ser usado como dados não rotulados.

2. **Modelo Base**  
   - Utilize um modelo de CNN simples ou uma arquitetura pré-definida (sugestão: ResNet-18) para a implementação.

3. **Implementação do FixMatch**
   - **Pseudo-rotulagem:** Implemente a geração de rótulos para os dados não rotulados usando predições de confiança de uma versão levemente aumentada da imagem.
   - **Consistência de Augmentation:** Aplique uma versão fortemente aumentada da imagem e treine a rede para manter consistência nos pseudo-rótulos.
   - **Função de Perda**:
     - O FixMatch utiliza uma função de perda híbrida, combinando a perda supervisionada e a não supervisionada:
       - **Perda Supervisionada:** Aplique a entropia cruzada entre os rótulos reais e as predições do modelo para os dados rotulados.
       - **Perda Não Supervisionada (Consistência de Pseudo-rótulos):** Para os dados não rotulados, aplique uma entropia cruzada entre os pseudo-rótulos e as previsões das imagens aumentadas, incluindo apenas as amostras com confiança acima de um limite predefinido (threshold).
       - A função de perda final é a soma ponderada das perdas supervisionada e não supervisionada.

   - **Detalhes Importantes nas Seções 2.3 e 2.4 do paper**;

4. **Treinamento e Otimização**

### 3. Experimentos e Análise

Para avaliar o desempenho do FixMatch, vocês devem realizar experimentos com diferentes quantidades de dados rotulados. Especificamente, testem com:

1. **1 rótulo por classe** (total de 10 rótulos): Este experimento extremo explora o desempenho do FixMatch com uma quantidade mínima de dados rotulados. Observem a eficácia da técnica de pseudo-rotulagem nesse cenário.

2. **4 rótulos por classe** (total de 40 rótulos): Com um conjunto pequeno, analisem o desempenho da rede com algumas amostras rotuladas e o impacto dos pseudo-rótulos.

3. **25 rótulos por classe** (total de 250 rótulos): Esse experimento permitirá uma análise mais profunda da eficácia do FixMatch em cenários com uma quantidade moderada de rótulos.

4. **400 rótulos por classe** (total de 4.000 rótulos): Avaliem o desempenho do modelo com um conjunto mais substancial de dados rotulados, investigando o impacto da quantidade crescente de rótulos.

**Proponha pelo menos mais algum teste, fundamente sua escolha e discuta os resultados.**

Para cada experimento:
   - Treine o modelo e avalie a acurácia nos dados de teste.
   - Documente os resultados e compare a eficácia do FixMatch com a quantidade de dados rotulados disponíveis.
   - Analise o impacto dos pseudo-rótulos na qualidade do modelo, principalmente nos cenários com poucos rótulos (1, 4 e 25 rótulos por classe).

### 4. Apresentação
No final, vocês devem preparar e apresentar:

1. Slides de apresentação ou relatório:
- Explicação da implementação de cada parte do FixMatch. (simples e rápida)
- Resultados e gráficos das avaliações para os quatro cenários de rótulos por classe. (Importante)
- Análise sobre o impacto da quantidade de dados rotulados, a função de perda híbrida, e o efeito dos thresholds e data augmentation. (Importante)

2. Apresentação de 10-15 minutos
- Grave uma apresentação do seu slide/relatório cobrindo todos os pontos pedidos.

*Note que a apresentação e o conteúdo dos slides deve cobrir todos os requisitos solicitados, pois sua avaliação vai depender 90% da apresentação. 



In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

from torch.optim import Adam
from torchvision.transforms import v2
from torchvision.datasets import CIFAR10
from torchvision.models import resnet18, ResNet18_Weights
from torchvision.io import decode_image
from PIL import Image

from tqdm import tqdm

from collections import defaultdict
from os.path import join
from itertools import cycle

from ctaugment import CTAugment

DEVICE = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
DEVICE

'cpu'

In [13]:
class LabeledDataset(Dataset):

    def __init__(self, dataset, indexes, device = "cpu"):
        self.dataset = dataset
        self.indexes = indexes
        self.device = device

        self.weak_augmentations = v2.Compose([
            v2.RandomHorizontalFlip(p=0.5),
            v2.RandomAffine(degrees=0, translate=(0.125, 0.125)), # translação em até 12,5% na vertical e horizontal
        ])

    def __len__(self):
        return len(self.indexes)

    def __getitem__(self, idx):
        #Pega o path da imagem e da label e carrega
        image, label = self.dataset[idx]
        image = image.to(torch.float).to(self.device)
        image = self.weak_augmentations(image) # Se acharmos útil podemos condicionar o augmentation ao treino e/ou à alguma probabilidade

        return image, label

    def get_original_image(self, idx):
        img_path = self.data[idx]
        return decode_image(img_path)



In [14]:
class UnlabeledDataset(Dataset):

    def __init__(self, dataset, indexes, device = "cpu", ctaugment = True):
        self.dataset = dataset
        self.indexes = indexes
        self.device = device
        
        self.weak_augmentations = v2.Compose([
            v2.RandomHorizontalFlip(p=0.5),
            v2.RandomAffine(degrees=0, translate=(0.125, 0.125)), # translação em até 12,5% na vertical e horizontal
        ])

        self.strong_augmentations = v2.Compose([
                CTAugment() if ctaugment else v2.RandAugment(),
                v2.RandomErasing(p=1, 
                                ratio=(1, 1), 
                                scale=(0.01, 0.01), 
                                value=127),
            ])

    def __len__(self):
        return len(self.indexes)

    def __getitem__(self, idx):
        #Pega o path da imagem e da label e carrega
        image, label = self.dataset[idx]
        image = image.to(torch.float).to(self.device)
        weak_image = self.weak_augmentations(image) # Mesma possiblididade de condicionar aqui também
        strong_image = self.strong_augmentations(image)
        return weak_image, strong_image

    def get_original_image(self, idx):
        img_path = self.data[idx]
        return decode_image(img_path)



In [15]:
def get_split_dataset(dataset, n_classes, n_samples, device = "cpu"):
    labeled_indexes = []

    total_indexes = n_classes * n_samples
    frequencies = defaultdict(lambda: 0)
    curr = 0

    while len(labeled_indexes) < total_indexes:
        if curr == len(dataset):
            raise RuntimeError("Não foi possível fazer split do dataset")

        label = dataset[curr][1]
        if frequencies[label] < n_samples:
            labeled_indexes.append(curr)
            frequencies[label] += 1
        curr += 1

    unlabeled_indexes = list(set(range(len(dataset))) - set(labeled_indexes))

    labeled_dataset = LabeledDataset(dataset, labeled_indexes, device)
    unlabeled_dataset = UnlabeledDataset(dataset, unlabeled_indexes, device)
    return labeled_dataset, unlabeled_dataset

In [16]:
#Exemplo:

base_dataset = CIFAR10("data", train = True, download = True)
labeled_dataset, unlabeled_dataset = get_split_dataset(base_dataset, 10, 5)
len(labeled_dataset), len(unlabeled_dataset), len(labeled_dataset) + len(unlabeled_dataset)

(50, 49950, 50000)

In [17]:
class ModelResnet18(nn.Module):
    def __init__(self, n_classes, device = "cpu"):
        super().__init__()
        self.device = device
        self.n_classes = n_classes

        self.resnet = resnet18(weights = ResNet18_Weights.IMAGENET1K_V1)
        self.resnet.fc = nn.Linear(in_features = 512, out_features = n_classes)
        self = self.to(device)

    def forward(self, x):
        return self.resnet(x)

In [18]:
modelo = ModelResnet18(10)
modelo

ModelResnet18(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, trac

In [19]:
class FixMatchLoss(nn.Module):
    def __init__(self, threshold, weight):
        super().__init__()
        self.threshold = threshold
        self.weight = weight
        self.cross_entropy = nn.CrossEntropyLoss()
        self.softmax = nn.Softmax(dim = 1)

    def forward(self, labeled_predictions, labeled_truth, unlabeled_weak_predictions, unlabeled_strong_predictions):
        l_s = self.cross_entropy(labeled_predictions, labeled_truth)

        with torch.no_grad():
            unlabeled_weak_predictions = self.softmax(unlabeled_weak_predictions)
            mask = unlabeled_weak_predictions.max(dim = 1)[0] > self.threshold
            if sum(mask) == 0:
                return l_s

        l_u = self.cross_entropy(unlabeled_strong_predictions[mask], unlabeled_weak_predictions[mask].argmax(dim = 1))

        return l_s + self.weight * l_u

In [20]:
N = 64
mu = 1.5
C = 10

mock_truth = torch.randint(C, size = (N,))
mock_preds = torch.normal(1.0, 5.0, size = (N, C))
mock_weak_preds = torch.normal(1.0, 5.0, size = (int(N * mu), C))
mock_strong_preds = torch.normal(1.0, 5.0, size = (int(N * mu), C))

loss = FixMatchLoss(0.7, 1)
loss(mock_preds, mock_truth, mock_weak_preds, mock_strong_preds)

tensor(16.8428)

In [21]:
class Trainer:
    def __init__(self, model, labeled_dataset, unlabeled_dataset, criterion, optimizer, val_dataset = None, device = "cpu"):
        self._model = model.to(device)

        self._labeled_dataset = labeled_dataset
        self._unlabeled_dataset = unlabeled_dataset
        self._val_dataset = val_dataset

        self._criterion = criterion
        self._optimizer = optimizer
        self._device = device
        print(device)

        self._history = {
            "train_loss": [],
            "val_acc": []
        }

    @property
    def history(self):
        return self._history
    
    def train_one_epoch(self, epoch):
        self._model.train()
        
        running_loss = 0.0
        zip_loaders = zip(cycle(self._labeled_dataloader), self._unlabeled_dataloader)
        progress_bar = tqdm(zip_loaders, desc = f"Epoch {epoch + 1} | Training", total = len(self._unlabeled_dataloader))

        for i, (labeled_data, unlabeled_data) in enumerate(progress_bar):
            # labeled_data = next(self._labeled_dataloader)
            # print(i, labeled_data[1].shape, labeled_data[1])

            labeled_images, labeled_labels = labeled_data
            labeled_images = labeled_images.to(self._device)
            labeled_labels = labeled_labels.to(self._device)

            unlabeled_weak, unlabeled_strong = unlabeled_data
            unlabeled_weak = unlabeled_weak.to(self._device)
            unlabeled_strong = unlabeled_strong.to(self._device)

            self._optimizer.zero_grad()

            labeled_predictions = self._model(labeled_images)
            unlabeled_weak_predictions = self._model(unlabeled_weak)
            unlabeled_strong_predictions = self._model(unlabeled_strong)

            loss = self._criterion(
                labeled_predictions,
                labeled_labels,
                unlabeled_weak_predictions,
                unlabeled_strong_predictions,
            )
            loss.backward()
            self._optimizer.step()

            running_loss += loss.item()
            progress_bar.set_postfix(loss = loss.item())

        epoch_loss = running_loss / len(self._unlabeled_dataloader)
        return epoch_loss

    def evaluate_accuracy(self, dataloader):
        self._model.eval()
        
        correct = 0
        total = 0
        
        with torch.no_grad():
            for images, labels in dataloader:
                images = images.to(torch.float).to(self._device)
                labels = labels.to(self._device)
                
                outputs = self._model(images)
                _, predicted = torch.max(outputs.data, 1) 
                
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        accuracy = 100 * correct / total
        return accuracy

    def fit(self, epochs, batch_size, mu=7):
        self._labeled_dataloader = DataLoader(self._labeled_dataset, batch_size=batch_size, shuffle=True)
        self._unlabeled_dataloader = DataLoader(self._unlabeled_dataset, batch_size = int(batch_size * mu), shuffle=True)
        if self._val_dataset:
            self._val_dataloader = DataLoader(self._val_dataset, batch_size = batch_size, shuffle=True)

        for epoch in range(epochs):
            train_loss = self.train_one_epoch(epoch)
            self._history["train_loss"].append(train_loss)
            
            if self._val_dataset:
                val_acc = self.evaluate_accuracy(self._val_dataloader)
                self.history["val_acc"].append(val_acc)
                print(f"val acc {val_acc}")

            print(f"Epoch {epoch + 1}: mean batch loss {train_loss}")

In [ ]:
n_classes = 10
n_samples = 10
epochs = 10

batch_size = 32
learning_rate = 0.001
threshold = 0.9
lambda_u = 1.0

base_dataset = CIFAR10("data", train=True, download=True, transform=v2.ToImage())
test_dataset = CIFAR10("data", train=False, download=True, transform=v2.ToImage())
labeled_dataset, unlabeled_dataset = get_split_dataset(base_dataset, n_classes, n_samples, DEVICE)

model = ModelResnet18(n_classes, DEVICE)

optimizer = Adam(model.parameters(), lr=learning_rate)
loss_function = FixMatchLoss(threshold, lambda_u)

trainer = Trainer(
    model,
    labeled_dataset,
    unlabeled_dataset,
    loss_function,
    optimizer,
    val_dataset=test_dataset,
    device=DEVICE
)

trainer.fit(epochs, batch_size)